In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import math
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: The Transpose Problem

### Memory Access Patterns

```
Matrix A (M×N, row-major):    Transposed A^T (N×M):
┌───┬───┬───┬───┐            ┌───┬───┬───┐
│ 0 │ 1 │ 2 │ 3 │   Memory:  │ 0 │ 4 │ 8 │
├───┼───┼───┼───┤   0,1,2,3  ├───┼───┼───┤
│ 4 │ 5 │ 6 │ 7 │   4,5,6,7  │ 1 │ 5 │ 9 │
├───┼───┼───┼───┤   8,9,...  ├───┼───┼───┤
│ 8 │ 9 │10 │11 │            │ 2 │ 6 │10 │
└───┴───┴───┴───┘            ├───┼───┼───┤
                             │ 3 │ 7 │11 │
                             └───┴───┴───┘

Naive transpose: out[j][i] = in[i][j]

Problem: Either reads OR writes are strided!
```

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
%%writefile transpose.cu
// transpose.cu - Naive vs Tiled Transpose
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// Naive transpose: coalesced reads, strided writes
__global__ void transpose_naive(const float* in, float* out, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;  // column
    int y = blockIdx.y * blockDim.y + threadIdx.y;  // row
    
    if (x < width && y < height) {
        // Read: coalesced (consecutive threads read consecutive addresses)
        // Write: strided (consecutive threads write addresses width apart)
        out[x * height + y] = in[y * width + x];
    }
}

// Tiled transpose using shared memory
__global__ void transpose_tiled(const float* in, float* out, int width, int height) {
    // Shared memory tile with extra column for bank conflict avoidance
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // Load tile from global memory (coalesced read)
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < width && (y + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(y + j) * width + x];
        }
    }
    
    __syncthreads();
    
    // Now x and y refer to transposed coordinates
    x = blockIdx.y * TILE_DIM + threadIdx.x;
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    // Write tile to global memory (coalesced write)
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < height && (y + j) < width) {
            // Note: read transposed from shared memory
            out[(y + j) * height + x] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

int main() {
    int width = 2048, height = 2048;
    size_t size = width * height * sizeof(float);
    
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    
    for (int i = 0; i < width * height; i++) h_in[i] = i;
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);
    
    dim3 block(TILE_DIM, BLOCK_ROWS);
    dim3 grid((width + TILE_DIM - 1) / TILE_DIM,
              (height + TILE_DIM - 1) / TILE_DIM);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark naive
    transpose_naive<<<grid, block>>>(d_in, d_out, width, height);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transpose_naive<<<grid, block>>>(d_in, d_out, width, height);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_ms;
    cudaEventElapsedTime(&naive_ms, start, stop);
    naive_ms /= 100;
    
    // Benchmark tiled
    transpose_tiled<<<grid, block>>>(d_in, d_out, width, height);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transpose_tiled<<<grid, block>>>(d_in, d_out, width, height);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float tiled_ms;
    cudaEventElapsedTime(&tiled_ms, start, stop);
    tiled_ms /= 100;
    
    // Calculate bandwidth
    double bytes = 2.0 * width * height * sizeof(float);  // read + write
    double naive_bw = bytes / (naive_ms * 1e6);   // GB/s
    double tiled_bw = bytes / (tiled_ms * 1e6);
    
    printf("Matrix: %d × %d\n", width, height);
    printf("Naive:  %.3f ms, %.1f GB/s\n", naive_ms, naive_bw);
    printf("Tiled:  %.3f ms, %.1f GB/s\n", tiled_ms, tiled_bw);
    printf("Speedup: %.2fx\n", naive_ms / tiled_ms);
    
    cudaFree(d_in); cudaFree(d_out);
    free(h_in); free(h_out);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o transpose transpose.cu
!./transpose

In [ ]:
# Naive transpose
@cuda.jit
def transpose_naive(input_matrix, output_matrix):
    """Naive transpose: coalesced reads, strided writes."""
    x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    height, width = input_matrix.shape
    
    if x < width and y < height:
        output_matrix[x, y] = input_matrix[y, x]

In [ ]:
# Test naive transpose
height, width = 1024, 1024
A = np.arange(height * width, dtype=np.float32).reshape(height, width)
B = np.zeros((width, height), dtype=np.float32)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)

block = (32, 8)
grid = ((width + 31) // 32, (height + 31) // 32)

transpose_naive[grid, block](d_A, d_B)
result = d_B.copy_to_host()

expected = A.T
print(f"Naive Transpose: {height}×{width}")
print(f"Correct: {'✓' if np.allclose(result, expected) else '✗'}")

---

## Part 2: Tiled Transpose with Shared Memory

In [ ]:
TILE_DIM = 32
BLOCK_ROWS = 8

@cuda.jit
def transpose_tiled(input_matrix, output_matrix):
    """Tiled transpose with coalesced reads AND writes."""
    # Shared memory with padding to avoid bank conflicts
    tile = cuda.shared.array((TILE_DIM, TILE_DIM + 1), dtype=np.float32)
    
    height, width = input_matrix.shape
    
    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    # Load tile (coalesced read)
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        if x < width and (y + j) < height:
            tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = input_matrix[y + j, x]
    
    cuda.syncthreads()
    
    # Swap block indices for output
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y
    
    # Store tile transposed (coalesced write)
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        if x < height and (y + j) < width:
            output_matrix[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j]

In [ ]:
# Test tiled transpose
height, width = 1024, 1024
A = np.arange(height * width, dtype=np.float32).reshape(height, width)
B = np.zeros((width, height), dtype=np.float32)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)

block = (TILE_DIM, BLOCK_ROWS)
grid = ((width + TILE_DIM - 1) // TILE_DIM, 
        (height + TILE_DIM - 1) // TILE_DIM)

transpose_tiled[grid, block](d_A, d_B)
result = d_B.copy_to_host()

expected = A.T
print(f"Tiled Transpose: {height}×{width}")
print(f"Correct: {'✓' if np.allclose(result, expected) else '✗'}")

---

## Part 3: Bank Conflict Analysis

In [ ]:
def explain_bank_conflicts():
    """Explain why bank conflicts occur in transpose."""
    print("Bank Conflicts in Matrix Transpose")
    print("=" * 50)
    print()
    print("Shared memory has 32 banks (bank = address % 32)")
    print()
    print("Without padding (TILE[32][32]):")
    print("  Storing: tile[ty][tx] - consecutive threads access different banks ✓")
    print("  Loading: tile[tx][ty] - all threads in column access SAME bank ✗")
    print()
    print("  Example column access (ty=0, different tx):")
    for tx in range(8):
        addr = tx * 32 + 0  # tile[tx][0]
        bank = addr % 32
        print(f"    Thread {tx}: tile[{tx}][0] → address {addr:3d} → bank {bank}")
    print("  All threads access bank 0 → 32-way bank conflict!")
    print()
    print("With padding (TILE[32][33]):")
    print("  Now tile[tx][ty] has stride 33 instead of 32")
    for tx in range(8):
        addr = tx * 33 + 0  # tile[tx][0] with 33-stride
        bank = addr % 32
        print(f"    Thread {tx}: tile[{tx}][0] → address {addr:3d} → bank {bank}")
    print("  All threads access different banks ✓")

explain_bank_conflicts()

---

## Part 4: Performance Comparison

In [ ]:
def benchmark_transpose(height, width, iterations=50):
    """Compare naive vs tiled transpose performance."""
    A = np.random.rand(height, width).astype(np.float32)
    B_naive = np.zeros((width, height), dtype=np.float32)
    B_tiled = np.zeros((width, height), dtype=np.float32)
    
    d_A = cuda.to_device(A)
    d_B_naive = cuda.to_device(B_naive)
    d_B_tiled = cuda.to_device(B_tiled)
    
    bytes_moved = 2 * height * width * 4  # Read + write
    
    # Naive
    block_naive = (32, 8)
    grid_naive = ((width + 31) // 32, (height + 31) // 32)
    
    transpose_naive[grid_naive, block_naive](d_A, d_B_naive)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(iterations):
        transpose_naive[grid_naive, block_naive](d_A, d_B_naive)
    cuda.synchronize()
    naive_ms = (time.perf_counter() - start) / iterations * 1000
    naive_bw = bytes_moved / (naive_ms * 1e6)  # GB/s
    
    # Tiled
    block_tiled = (TILE_DIM, BLOCK_ROWS)
    grid_tiled = ((width + TILE_DIM - 1) // TILE_DIM, 
                  (height + TILE_DIM - 1) // TILE_DIM)
    
    transpose_tiled[grid_tiled, block_tiled](d_A, d_B_tiled)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(iterations):
        transpose_tiled[grid_tiled, block_tiled](d_A, d_B_tiled)
    cuda.synchronize()
    tiled_ms = (time.perf_counter() - start) / iterations * 1000
    tiled_bw = bytes_moved / (tiled_ms * 1e6)  # GB/s
    
    return naive_ms, naive_bw, tiled_ms, tiled_bw

print(f"{'Size':<15} {'Naive (ms)':<12} {'Naive BW':<12} {'Tiled (ms)':<12} {'Tiled BW':<12} {'Speedup':<10}")
print("=" * 85)

for size in [512, 1024, 2048]:
    try:
        n_ms, n_bw, t_ms, t_bw = benchmark_transpose(size, size)
        speedup = n_ms / t_ms
        print(f"{size}×{size:<10} {n_ms:<12.3f} {n_bw:<12.1f} {t_ms:<12.3f} {t_bw:<12.1f} {speedup:.2f}x")
    except Exception as e:
        print(f"{size}×{size:<10} Error: {e}")

---

## Part 5: Memory Access Visualization

In [ ]:
def visualize_access_patterns():
    """Visualize memory access patterns for transpose."""
    print("Memory Access Pattern Analysis")
    print("=" * 60)
    
    print("\n1. NAIVE TRANSPOSE")
    print("   Thread arrangement: blockDim = (32, 8)")
    print("   Each thread handles one element")
    print()
    print("   READ (coalesced):")
    print("   Warp threads 0-31 read in[y][0], in[y][1], ..., in[y][31]")
    print("   → Addresses: base, base+4, base+8, ... (consecutive) ✓")
    print()
    print("   WRITE (strided):")
    print("   Warp threads 0-31 write out[0][y], out[1][y], ..., out[31][y]")
    print("   → Addresses: base, base+H*4, base+2*H*4, ... (stride H) ✗")
    
    print("\n2. TILED TRANSPOSE")
    print("   Step 1 - READ (coalesced):")
    print("   Same as naive - warp reads consecutive addresses ✓")
    print()
    print("   Step 2 - STORE to shared (no bank conflict):")
    print("   tile[ty][tx] - each thread writes different bank ✓")
    print()
    print("   Step 3 - LOAD from shared (no bank conflict with padding):")
    print("   tile[tx][ty+j] with TILE[32][33] padding ✓")
    print()
    print("   Step 4 - WRITE (coalesced):")
    print("   Swapped block indices mean consecutive threads write")
    print("   consecutive global memory addresses ✓")

visualize_access_patterns()

---

## Exercises

### Exercise 1: In-place Transpose

In [ ]:
# TODO: Implement in-place transpose for square matrices
# Hint: Only transpose above diagonal, swap with below diagonal

@cuda.jit
def transpose_inplace(matrix):
    """In-place transpose for square matrix."""
    pass  # Your implementation

### Exercise 2: Batched Transpose

In [ ]:
# TODO: Implement batched transpose for 3D tensor
# Shape: (batch, height, width) → (batch, width, height)

@cuda.jit
def transpose_batched(input_3d, output_3d):
    """Batched transpose: transpose each 2D slice."""
    pass  # Your implementation

---

## Summary

### Key Insights

| Approach | Reads | Writes | Performance |
|----------|-------|--------|-------------|
| Naive | Coalesced ✓ | Strided ✗ | Low bandwidth |
| Tiled | Coalesced ✓ | Coalesced ✓ | ~Peak bandwidth |

### CUDA C++ Pattern

```cpp
// Key: Extra column for bank conflict avoidance
__shared__ float tile[TILE_DIM][TILE_DIM + 1];

// Load: row-major from global
tile[ty][tx] = in[y * width + x];
__syncthreads();

// Store: column-major to global (but looks row-major after swap)
out[(newY) * height + newX] = tile[tx][ty];
```

### Bank Conflict Solution

```
Without padding: tile[32][32] → column access = 32-way conflict
With padding:    tile[32][33] → stride 33 → no conflicts
```

### Tomorrow: cuBLAS Integration
We'll use the optimized cuBLAS library for matrix operations.